In [1]:
# To capture the NNP library stdout output we have two possible options:
#
# 1.) Install "wurlitzer" available via pip or github.com/minrk/wurlitzer
#     > pip install wurlitzer
#     and load as extension
#     > %load_ext wurlitzer
#
# 2.) Silence the library output (which will otherwise go to Jupyter server shell)
#     > nnp = pynnp.Mode()
#     > nnp.log.writeToStdout = False
#     > ... do initialization ...
#     and recall the log later:
#     > for line in nnp.log.getLog():
#     >     sys.stdout.write(line)
#
# Option 1.) is obviously more comfortable and will also capture output to stderr.
%load_ext wurlitzer
import pynnp
from sys import stdout as so

ImportError: No module named pynnp

In [ ]:
# Create instance of NNP setup.                               
nnp = pynnp.Mode()

# Turn off output to stdout (for Option 2.) described above).
#nnp.log.writeToStdout = False

In [ ]:
# Run initialization steps (symmetry functions only).
nnp.initialize()                                                                  
nnp.loadSettingsFile("input.nn")                                                  
nnp.setupElementMap()                                                             
nnp.setupElements()                                                               
nnp.setupCutoff()                                                                 
nnp.setupSymmetryFunctions()                                                      
nnp.setupSymmetryFunctionGroups()
# Either use symmetry function scaling...
#nnp.setupSymmetryFunctionScaling("scaling.data")                                  
#nnp.setupSymmetryFunctionStatistics(False, False, True, False)
# ... or calculate raw values.
nnp.setupSymmetryFunctionScalingNone()

# Print log file (for Option 2.) described above).
#for line in m.log.getLog():
#    so.write(line)

In [ ]:
# Create an empty structure.
struct = pynnp.Structure()

# Tell the structure which elements are to be expected,
# i.e. pass the ElementMap from the NNP setup instance.
# Note: this step is required!
struct.setElementMap(nnp.elementMap)

In [ ]:
# Read in configuration from file.
# WARNING: Do not read repeatedly as this will pile up atoms!
struct.readFromFile("input.data.1")

# Print some information:
print "Number of atoms: ", struct.numAtoms
print "Number of atoms per element: "
for (i, n) in enumerate(struct.numAtomsPerElement):
    print nnp.elementMap[i], ": ", n

In [ ]:
# Retrieve cutoff radius form NNP setup.                                        
cutoffRadius = nnp.getMaxCutoffRadius()                                           
print "Cutoff radius = ", cutoffRadius                                          
                                                                                
# Calculate neighbor list.                                                      
struct.calculateNeighborList(cutoffRadius)

# Show some information about neighbor list.
for atom in struct.atoms:
    print "Atom {0:4d} ({1:2s}): {2:3d} neighbors".format(atom.index, nnp.elementMap[atom.element], atom.numNeighbors)

In [ ]:
# Calculate symmetry functions for all atoms (using groups is generally faster).
#nnp.calculateSymmetryFunctions(s, False)
nnp.calculateSymmetryFunctionGroups(struct, False)
                                                                                
# Retrieve symmetry functions of atom with index 0.
# Symmetry functions sort order is provided in the library output
# of the setupSymmetryFunctions() method (see above).
print struct.atoms[0].G

In [ ]:
# Repeat the procedure for another structure.

# First reset the structure, this will erase all data except for the ElementMap.
struct.reset()

struct.readFromFile("input.data.2")
struct.calculateNeighborList(cutoffRadius)
%time nnp.calculateSymmetryFunctionGroups(struct, False)
print struct.atoms[0].G